In [1]:
# import dependencies

In [2]:
import pandas as pd
from openpyxl import load_workbook

import numpy as np

import schedule
import time

from datetime import datetime

import plotly
import plotly.graph_objects as go
import plotly.express as px

import chart_studio

import dash
import dash_html_components as html
import dash_core_components as dcc

import cufflinks as cf

cf.go_offline()

In [3]:
chart_studio.tools.set_credentials_file(username = 'thedjamba', api_key = '***********')

In [4]:
# get data from web

In [9]:
def get_history():
    bg = pd.read_excel("Bulgaria.xlsx")
    bg['Дата'] = bg['Дата'].dt.date
    bg['болни'] = bg['потвърдени'] - bg['смъртни'] - bg['възстановени']
    bg['нови потвърдени'] = bg.groupby('Държава')['потвърдени'].diff()
    bg['нови смъртни'] = bg.groupby('Държава')['смъртни'].diff()
    bg['нови възстановени'] = bg.groupby('Държава')['възстановени'].diff()
    bg['Дата'] = bg['Дата'].apply(lambda x: x.strftime('%Y-%m-%d'))
    return(bg)

In [10]:
url = "http://www.mh.government.bg/bg/informaciya-za-grazhdani/potvrdeni-sluchai-na-koronavirus-na-teritoriyata-na-r-blgariya/"

def get_today():
    today = pd.read_html(url)[0]
    today = pd.pivot_table(today, values = 1, columns = 0)
    today.columns = ['възстановени', 'смъртни', 'потвърдени']
    today['болни'] = today['потвърдени'] - today['смъртни'] - today['възстановени']
    today['Дата'], today['Държава'], today['събитие'] = [datetime.today().strftime('%Y-%m-%d'), 'България', np.nan]
    today = today[['Дата', 'Държава', 'потвърдени', 'смъртни', 'възстановени', 'събитие', 'болни']]
    return(today)

In [11]:
get_today()

,Дата,Държава,потвърдени,смъртни,възстановени,събитие,болни
1,2020-04-11,България,661,28,62,NaN,571


In [12]:
trace0 = go.Scatter (x = get_history()['Дата'], y = get_history()['болни'], name = 'болни', line = dict(width = 7))
trace1 = go.Scatter (x = get_history()['Дата'], y = get_history()['смъртни'], name = 'смъртни', line = dict(width = 7))
trace2 = go.Scatter (x = get_history()['Дата'], y = get_history()['възстановени'], name = 'възстановени', line = dict(width = 7))

data = [trace0, trace1, trace2]
layout = dict(title= 'COVID19 Разпространение в България', xaxis = dict(), yaxis = dict())
fig= dict(data = data, layout = layout)

In [13]:
cf.iplot(fig, filename = 'COVID19 Разпространение в България')

In [15]:
bg_melt = get_history().melt(id_vars='Дата', value_vars=['болни', 'смъртни', 'възстановени'])
bg_melt.columns = ['Дата', 'Случаи', 'Брой']

In [16]:
maxi = get_history()['потвърдени'].max()

In [17]:
def roundup(x):
    return x if x % 100 == 0 else x + 100 - x % 100

In [18]:
fig = px.bar(bg_melt, x="Случаи", y="Брой", color="Случаи",
             animation_frame="Дата", range_y=[0, roundup(maxi)])

In [19]:
cf.iplot(fig, filename = 'COVID19 анимация')

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    html.Div([
        html.Div([
            #html.H3('Column 1'),
            dcc.Graph(figure= dict(data = data, layout = layout)),
        ], className="six columns"),

        html.Div([
            #html.H3('Column 2'),
            dcc.Graph(figure = px.bar(bg_melt, x="Случаи", y="Брой", color="Случаи", animation_frame="Дата", range_y=[0,roundup(maxi)]))
        ], className="six columns"),
    ], className="row")
])

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
C:\Users\ioan.djambov\AppData\Local\Continuum\anaconda3\lib\site-packages\dash\resources.py:49: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.ly/external-resources

127.0.0.1 - - [11/Apr/2020 22:18:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 22:18:07] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 22:18:07] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 22:18:07] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 22:18: